In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
def load_data(filepath):
    import h5py
    import numpy as np
    h5f = h5py.File(filepath, 'r')
    X = h5f['X'][:]
    classnames = [s.decode('utf-8') for s in h5f['classname'][:]]
    filenames = [s.decode('utf-8') for s in h5f['filename'][:]]
    h5f.close()
    return X, np.array(classnames), np.array(filenames)

def create_train_test_idx(classnames):
    import pandas as pd
    df = pd.DataFrame(data=enumerate(classnames), columns=['index', 'classname'])
    test_df = None
    train_df = None
    for classname in df['classname'].unique():
        test_tmp_df = df[df['classname']==classname].sample(50, replace=False, random_state=1234)
        train_tmp_df = df[(df['classname']==classname) & ~(df['index'].isin(test_tmp_df['index']))]
        test_df = test_tmp_df if test_df is None else pd.concat([test_df, test_tmp_df])
        train_df = train_tmp_df if train_df is None else pd.concat([train_df, train_tmp_df])    
    return train_df['index'].values, test_df['index'].values
    
X, classnames, filenames = load_data('data/RockAI_images_224x224.h5')
train_idx, test_idx = create_train_test_idx(classnames)
X_train, X_test = X[train_idx], X[test_idx]
y_train = [classnames[i] for i in train_idx]
y_test = [classnames[i] for i in test_idx]

num_classes = 2

#Pre-process the data
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)
y_train = [0 if x=='No_RA' else 1 for x in y_train]
y_test = [0 if x=='No_RA' else 1 for x in y_test]
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [4]:
import numpy as np

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

datagen = ImageDataGenerator(
   get_random_eraser(v_l=0, v_h=1, pixel_level=True),
   shear_range=0.2,
   zoom_range=0.2,
   rotation_range = 30,
   horizontal_flip=True,
   vertical_flip=True)
# datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
# datagen = ImageDataGenerator()
datagen.fit(X_train)

In [5]:
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
inception_model.trainable = False

model = Sequential()
model.add(inception_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(num_classes, activation='softmax'))

# tf.keras.utils.plot_model(model, show_shapes=True, to_file='outdir/inception_model.png')
# model.summary()

In [6]:
batch_size = 32
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

historytemp = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=30)

Epoch 1/30
42/42 [==============================] - 11s 170ms/step - loss: 0.5224 - accuracy: 0.9311
Epoch 2/30
42/42 [==============================] - 7s 163ms/step - loss: 0.2065 - accuracy: 0.9614
Epoch 3/30
42/42 [==============================] - 7s 165ms/step - loss: 0.1937 - accuracy: 0.9522
Epoch 4/30
42/42 [==============================] - 7s 164ms/step - loss: 0.1610 - accuracy: 0.9449
Epoch 5/30
42/42 [==============================] - 7s 164ms/step - loss: 0.1540 - accuracy: 0.9529
Epoch 6/30
42/42 [==============================] - 7s 163ms/step - loss: 0.1261 - accuracy: 0.9597
Epoch 7/30
42/42 [==============================] - 7s 165ms/step - loss: 0.1572 - accuracy: 0.9511
Epoch 8/30
42/42 [==============================] - 7s 165ms/step - loss: 0.1452 - accuracy: 0.9512
Epoch 9/30
42/42 [==============================] - 7s 164ms/step - loss: 0.1268 - accuracy: 0.9515
Epoch 10/30
42/42 [==============================] - 7s 164ms/step - loss: 0.1471 - accuracy: 0.950

In [7]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 1s 178ms/step - loss: 0.6369 - accuracy: 0.7300


[0.6369341015815735, 0.7300000190734863]